In [1]:
import sys
import os

current_dir = os.getcwd()

sys.path.append(os.path.abspath(os.path.join(current_dir, '..')))

In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from datetime import datetime, timedelta
from models.Level import Level
from models.MemberLevelScore import MemberLevelScore
from models.MemberProductAccount import MemberProductAccount
from models.LevelsFull import LevelsFull, LevelData, Movement
from models.StandardChartData import StandardChartData, StandardDataPoint
from models.Timeline import Timeline

In [7]:
# Cell 4: Load DataFrames

levels = pd.read_csv("../data/levels.csv")
member_level_scores = pd.read_csv("../data/member_level_scores.csv")
member_level_scores_history = pd.read_csv("../data/member_level_scores_history.csv")
member_product_accounts = pd.read_csv("../data/member_product_accounts.csv")

print("Levels DataFrame:")
print(levels.head())

print("\nMember Level Scores DataFrame:")
print(member_level_scores.head())

print("\nMember Level Scores History DataFrame:")
print(member_level_scores_history.head())

print("\nMember Product Accounts DataFrame:")
print(member_product_accounts.head())


Levels DataFrame:
  client_account_id                timestamp  \
0        federal-cu  2024-10-25 12:19:50.988   
1        federal-cu  2024-10-25 12:19:50.988   
2        federal-cu  2024-10-25 12:19:50.988   
3        federal-cu  2024-10-25 12:19:50.988   
4        federal-cu  2024-10-25 12:19:50.988   

                               level_id level_name  level_score_start  \
0  54f4780e-5f82-4683-a9e8-146511392625          A                 90   
1  9167207b-6ae0-4e0a-b26d-5a53eb1a306f          B                 72   
2  d27ae886-bd8d-498e-a388-5911ed804938          C                 54   
3  88a94050-c4df-47d3-a1b0-e8b772a6ef16          D                 36   
4  54c3259c-cb7d-4426-8797-3a3a200743ca          E                 18   

   level_score_end  
0              100  
1               90  
2               72  
3               54  
4               36  

Member Level Scores DataFrame:
  client_account_id                timestamp  member_id level_score_type  \
0        federal-cu 

In [10]:
# Cell 5: Preprocessing

# Convert member_id to string for consistency across DataFrames.
for df in [levels, member_level_scores, member_level_scores_history, member_product_accounts]:
    if 'member_id' in df.columns:
        df['member_id'] = df['member_id'].astype(str)

# Convert date fields in member_level_scores.
if 'score_date' in member_level_scores.columns:
    member_level_scores['score_date'] = pd.to_datetime(member_level_scores['score_date'], errors='coerce')
if 'timestamp' in member_level_scores.columns:
    member_level_scores['timestamp'] = pd.to_datetime(member_level_scores['timestamp'], errors='coerce')

# Convert date fields in member_level_scores_history.
member_level_scores_history['score_date'] = pd.to_datetime(member_level_scores_history['score_date'], errors='coerce')

# Debug: Check types and a summary.
print("Member Level Scores dtypes:")
print(member_level_scores.dtypes)
print("\n")
print("Member Level Scores History dtypes:")
print(member_level_scores_history.dtypes)


Member Level Scores dtypes:
client_account_id            object
timestamp            datetime64[ns]
member_id                    object
level_score_type             object
level_score                 float64
score_date           datetime64[ns]
active_member                  bool
dtype: object


Member Level Scores History dtypes:
client_account_id            object
timestamp                    object
member_id                    object
level_score_type             object
level_score                 float64
score_date           datetime64[ns]
active_member                  bool
dtype: object


In [ ]:
# Cell 6: Process Current Member Scores and Assign Levels

# Creating copy of member_level_scores dataframe as the dataframe will be manipulated later
current_scores = member_level_scores.copy()
current_scores['level_score'] = current_scores['level_score'].fillna(-1)

# Sort levels by score (lowest threshold to highest threshold)
levels = levels.sort_values('level_score_start').reset_index(drop=True)
level_order = {level: i for i, level in enumerate(levels['level_name'])}

# Create an IntervalIndex for level boundaries (using left-closed/left-inclusive intervals)
levels_intervals = pd.IntervalIndex.from_arrays(
    levels['level_score_start'],
    levels['level_score_end'],
    closed='left'
)

level_labels = levels['level_name'].astype(str).tolist()

# Use pd.cut to assign levels to current_scores
current_scores['current_level'] = pd.cut(
    current_scores['level_score'],
    bins=levels_intervals,
    labels=level_labels,
    include_lowest=True,
    right=False
)
# Applying level_names as labels for each interval
current_scores['current_level'] = current_scores['current_level'].cat.rename_categories(level_labels)

# Map the current_level to a numeric index
# Indices are used to track member movement from one level to the next
current_scores['current_level_index'] = current_scores['current_level'].map(level_order)

print("Current Scores after Level Assignment:")
print(current_scores[['member_id', 'level_score', 'current_level', 'current_level_index']].head(50))


Current Scores after Level Assignment:
   member_id  level_score current_level current_level_index
0       1769    42.885418             D                   2
1       1785    11.784956             F                   0
2       1849    23.128963             E                   1
3       1857    43.876155             D                   2
4       1881    16.480550             F                   0
5       1913    27.275888             E                   1
6       1917    16.020929             F                   0
7       1937    10.168275             F                   0
8       1973    33.288255             E                   1
9       1989    20.525269             E                   1
10      2041    53.698749             D                   2
11      2077    63.340284             C                   3
12      2093    15.493370             F                   0
13      2101    27.919605             E                   1
14      2133    16.726753             F                   0
1

In [31]:
# Cell 7: Current Date and Timeline Offsets

# Determine the "current" date using the max timestamp if available
if 'timestamp' in current_scores.columns and current_scores['timestamp'].notnull().any():
    current_date = current_scores['timestamp'].max()
else:
    current_date = pd.Timestamp(datetime.now())

print("Current Date:", current_date)

# Define timeline offsets in days
timeline_offsets = {
    Timeline.OneMonth.value: 30,
    Timeline.ThreeMonths.value: 90,
    Timeline.SixMonths.value: 180,
    Timeline.TwelveMonths.value: 365,
    Timeline.YearToDate.value: (current_date - pd.Timestamp(current_date.year, 1, 1)).days
}

print("Timeline Offsets (in days):", timeline_offsets)


Current Date: 2024-11-09 00:00:00
Timeline Offsets (in days): {'1m': 30, '3m': 90, '6m': 180, '12m': 365, 'ytd': 313}


In [ ]:
# Cell 8: Historical Counts and Movement Metrics

# Dictionaries to store historical member counts and movement
history_counts = {level: {} for level in levels['level_name']}
movement_counts = {level: {} for level in levels['level_name']}

# Iterate thru each timeline checkpoint
for timeline_val, offset in timeline_offsets.items():
    if timeline_val == Timeline.YearToDate.value:
        cutoff_date = pd.Timestamp(current_date.year, 1, 1)
    else:
        cutoff_date = current_date - pd.Timedelta(days=offset)
    
    # Get historical records up to the cutoff date
    hist_before_cutoff = member_level_scores_history[member_level_scores_history['score_date'] <= cutoff_date]
    if hist_before_cutoff.empty:
        for level in levels['level_name']:
            history_counts[level][timeline_val] = 0
            movement_counts[level][timeline_val] = {'growth': 0, 'churn': 0}
        continue
    
    # For each member, get latest record at or before the cutoff
    hist_latest = hist_before_cutoff.sort_values('score_date').groupby('member_id', as_index=False).last()
    # Use level score intervals on historical scores
    hist_latest['historical_level'] = pd.cut(
        hist_latest['level_score'],
        bins=levels_intervals,
        labels=level_labels,
        include_lowest=True,
        right=False
    )
    hist_latest['historical_level'] = hist_latest['historical_level'].cat.rename_categories(level_labels)
    hist_latest['historical_level_index'] = hist_latest['historical_level'].map(level_order)
    
    # Merge current and historical data on member_id
    merged = pd.merge(current_scores, hist_latest[['member_id', 'historical_level', 'historical_level_index']], on='member_id', how='inner')
    
    # Handle missing historical indices with the current index
    # In this case, historical level indices are being filled with the current level indices, but dropping the row would result in the same outcome
    # Ultimately, we must assume that the user did not move up or down any levels if we are met with NaN data for index
    merged['historical_level_index'] = merged['historical_level_index'].fillna(merged['current_level_index'])
    
    # For each level, compute historical count and movement
    for level in levels['level_name']:

        # Getting total members in which their latest historical level is the one being checked in the loop
        hist_count = (hist_latest['historical_level'] == level).sum()
        history_counts[level][timeline_val] = int(hist_count)
        
        # Filtering to only get members at level being checked by the loop
        current_in_level = merged[merged['current_level'] == level]
        
        # Compare members' current level with their historical level from x time ago (timeline values)
        growth = (current_in_level['historical_level_index'].astype(int) < current_in_level['current_level_index'].astype(int)).sum()
        churn = (current_in_level['historical_level_index'].astype(int) > current_in_level['current_level_index'].astype(int)).sum()
        movement_counts[level][timeline_val] = {'growth': int(growth), 'churn': int(churn)}
        
print("\nHistorical Counts by Level and Timeline:")
print(history_counts)
print("\nMovement Counts by Level and Timeline:")
print(movement_counts)



Timeline 1m: Cutoff Date = 2024-10-10 00:00:00
Historical records sample:
  member_id client_account_id                timestamp level_score_type  \
0    100001        federal-cu  2024-10-05 00:00:00.000           MPP V4   
1   1000037        federal-cu  2024-10-05 00:00:00.000           MPP V4   
2   1000053        federal-cu  2024-10-05 00:00:00.000           MPP V4   
3    100009        federal-cu  2024-10-05 00:00:00.000           MPP V4   
4   1000101        federal-cu  2024-10-05 00:00:00.000           MPP V4   

   level_score score_date  active_member historical_level  \
0    47.739508 2024-10-05           True                D   
1    10.332937 2024-10-05           True                F   
2    40.791435 2024-10-05           True                D   
3    43.097296 2024-10-05           True                D   
4    33.945536 2024-10-05           True                E   

  historical_level_index  
0                      2  
1                      0  
2                      2  

In [46]:
# Cell 9: Assemble LevelData and LevelsFull

level_data_list = []
for i, level_row in levels.iterrows():
    level_name = level_row['level_name']
    score_start = level_row['level_score_start']
    score_end = level_row['level_score_end']
    
    # Current member count for this level
    curr_members = current_scores[current_scores['current_level'] == level_name]
    member_count = curr_members['member_id'].nunique()
    
    # Average product count for members in this level
    if member_count > 0:
        total_products = member_product_accounts[member_product_accounts['member_id'].isin(curr_members['member_id'])].shape[0]
        avg_product_count = round(total_products / member_count)   # Rounded to nearest whole number
    else:
        avg_product_count = 0
    
    # Building member_count_history chart
    history_points = []
    for timeline_val, count in history_counts[level_name].items():
        history_points.append(StandardDataPoint(key=timeline_val, value=count))
    chart_data = StandardChartData(points=history_points)
    
    # Building Movement objects
    movements = []
    for timeline_val, mv in movement_counts[level_name].items():
        try:
            timeline_enum = Timeline(timeline_val)
        except ValueError:
            continue
        movements.append(Movement(timeline=timeline_enum.value, growth=mv['growth'], churn=mv['churn']))
    
    # Final LevelData assembly
    level_data = LevelData(
        level=level_name,
        member_count=member_count,
        score_start=score_start,
        score_end=score_end,
        avg_product_count=avg_product_count,
        member_count_history=chart_data,
        movement=movements
    )
    level_data_list.append(level_data)

levels_full_metric = LevelsFull(levels=level_data_list)

print("\nFinal LevelsFull Object:")
print(levels_full_metric)



Final LevelsFull Object:
LevelsFull(levels=[LevelData(level='F', member_count=22681, score_start=0, score_end=18, avg_product_count=2, member_count_history=StandardChartData(points=[StandardDataPoint(key='1m', value=22604), StandardDataPoint(key='3m', value=21777), StandardDataPoint(key='6m', value=20380), StandardDataPoint(key='12m', value=17793), StandardDataPoint(key='ytd', value=18414)]), movement=[Movement(timeline='1m', growth=0, churn=802), Movement(timeline='3m', growth=0, churn=1583), Movement(timeline='6m', growth=0, churn=2822), Movement(timeline='12m', growth=0, churn=5014), Movement(timeline='ytd', growth=0, churn=4345)]), LevelData(level='E', member_count=63322, score_start=18, score_end=36, avg_product_count=3, member_count_history=StandardChartData(points=[StandardDataPoint(key='1m', value=63274), StandardDataPoint(key='3m', value=62785), StandardDataPoint(key='6m', value=62116), StandardDataPoint(key='12m', value=62074), StandardDataPoint(key='ytd', value=61618)]), mo